# Final Exam

## FINM 37500 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

# Instructions

## Please note the following:

Points
* You have **180 minutes** to complete the exam.
* For every minute late you submit the exam, you will lose one point.
Final Exam

Submission
* You will upload your solution to the Final Exam assignment on Canvas, where you downloaded this.
* Your submission should be readable, (the graders can understand your answers,) and it should include all code used in your analysis in a file format that the code can be executed. (ie. .ipynb preferred, .pdf is unacceptable.)

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation, (only for this exam.)

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

Answer Quality
* For conceptual questions, note that we will grade your answer for its relevance to our course focus and discussion. 
* Making points that are irrelevant, out-of-context, or overly general will may not help and could hurt your score for the question.
* This may be particularly relevant for answers which are copied from LLMs such as ChatGPT, but do not get at the heart of our contextual course learning.

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 20     |
| 2       | 25     |
| 3       | 50     |
| 4       | 55     |
| **Total**   | **150**|

#### Each numbered question is worth 5pts unless otherwise noted.

***

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `exam_data_2024-03-05.xlsx`

This file has sheets for...
* curve data - discount factors and forward volatilities
* BDT tree of rates (continuously compounded, as usual)
* vol quotes (across strikes) on swaptions for a particular expiry and tenor
* SABR parameters for a vol skew of the expiry and tenor associted with the vol quotes on the previous sheet.

Note
* the curve data is given at quarterly frequency
* all rates reported in the curve data are quarterly compounded, which is conveneint for the study of quarterly products in this exam (whether caps, floors, swaps, etc.)

### Load Data

If useful, the following code loads the data:

In [67]:
import numpy as np
import pandas as pd

from datetime import date
from datetime import datetime

import sys
sys.path.insert(0, '../cmds')
from ficcvol import *
from binomial import *
from ratecurves import *
from volskew import *
#from treasury_cmds import compound_rate

In [31]:
DATE = '2024-03-05'

FILEIN = f'../data/exam_data_{DATE}.xlsx'

sheet_curves = 'rate curves'
sheet_tree = 'rate tree'
sheet_volskew = 'bloomberg vcub'
sheet_sabrparams = 'sabr params'

curves = pd.read_excel(FILEIN, sheet_name=sheet_curves).set_index('tenor')

ratetree = pd.read_excel(FILEIN, sheet_name=sheet_tree).set_index('state')
ratetree.columns.name = 'time'

volskew = pd.read_excel(FILEIN, sheet_name=sheet_volskew)

sabrparams = pd.read_excel(FILEIN, sheet_name=sheet_sabrparams).set_index('parameter')

***

# 1. Models

## 1.1. (10pts)

Be specific as to which type of data is used to estimate a binomial rate tree. Or, if the feature is assumed rather than estimated, make that clear.

How do we estimate the rate tree's...
* drift
* distribution (Ho-Lee vs Black-Derman-Toy)
* nodes
* probabilities

**ANSWER:** 

drift: We estimate drift utilizing forward vols and discount rates. The process is a two parameter fitting where given the forward vols (or potentially calcualted from falt vols, etc.) we can fit the drift (theta) such that it fits the discount rates. This is done by recursively solving for the drifts for each time period by fitting the term srtucure of the rates.

distribtuion: Both BDT and Ho-Lee can be used to estimate a binomial rate tree. Both use the drifts, vols to estiamte th ebinomial tree. The difference is that BDT is log normal and Ho-Lee is normal.

Nodes: The nodes are given from the structure of the security we are trying to model and our step size.

Probabilities: The probaiblies are given as p* = 0.5 and instead the up and down movement values are adapted

## 1.2.

Consider an interest-rate cap with only a single caplet. Which of the following could we use as an underlying binomial tree to price this cap?

* interest-rate tree
* bond tree
* swap tree
* floor tree

**ANSWER:** 

We could use an interest rate tree as a caplet is essentially a call option on an interrest rate. Therefore we could model an interest rate tree and then adjust the tree according to the payoff (max(r-k, 0))

## 1.3.

True or False: SABR is used to have structure in modeling implied volatilities across strikes, tenors, and expirations.

**ANSWER:**

True. SABR is used to model changing implied volatilities and to define a process for implied volatility. It is providding strucutre in the sense that it is doing more than just providinga function for the implied vol as it defines a stochastic process for how the implied vol evolves over time.

***

# 2. Pricing with BDT and Black

Use `rate curves` for market data.

## 2.1.

Recall that a floor is a portfolio of floorlets which
* depend on the realized interest rate one quarter before the expiration.
* each have the same strike interest rate.

Use Black's formula to price **just one floorlet**
* expiring at `T=3`
* struck at the `T=3` swap rate
* notional of `100`

In [32]:
curves

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052226,0.052226,0.987112,NaN,NaN,0.140538
0.50,0.051470,0.051465,0.974756,0.050705,0.140538,0.140538
0.75,0.050427,0.050411,0.963125,0.048303,0.161925,0.178617
1.00,0.049243,0.049209,0.952268,0.045606,0.183311,0.215305
1.25,0.047534,0.047467,0.942722,0.040506,0.233815,0.353890
1.50,0.046112,0.046016,0.933672,0.038769,0.272932,0.377696
1.75,0.044914,0.044793,0.925009,0.037462,0.302241,0.393585
2.00,0.043913,0.043770,0.916617,0.036620,0.323320,0.398881
2.25,0.042897,0.042729,0.908800,0.034407,0.337748,0.395042


In [33]:
floor_curves = flat_to_forward_vol(curves)
floor_curves

,flat vols,fwd vols,cap prices
tenor,,,
0.25,NaN,NaN,NaN
0.50,0.140538,0.140538,0.026355
0.75,0.161925,0.178617,0.075733
1.00,0.183311,0.215305,0.150898
1.25,0.233815,0.353890,0.291985
1.50,0.272932,0.377696,0.458120
1.75,0.302241,0.393585,0.641719
2.00,0.323320,0.398881,0.835751
2.25,0.337748,0.395042,1.033555


In [34]:
N = 100
T = 3
dt = 1/4
STRIKE = curves.loc[3, 'swap rates']
DISCOUNT = curves.loc[3, 'discounts']
VOL = floor_curves.loc[3, 'fwd vols']
F = curves.loc[2.75, 'spot rates']
compound = 4
tsteps = int(T/dt)

In [35]:
P = blacks_formula(3, VOL,  STRIKE, F, DISCOUNT, isCall=False)
P

0.009166593971569133

## 2.2.

Use Black's formula to price the entire floor with expiration of `T=3`.

The floor has floorlets at `quarterly` frequency, except the first quarter.

In [36]:
N * dt * P

0.22916484928922834

## 2.3.

Use `rate tree` for a BDT binomial tree fit to market data for your convenience.

Use this binomial tree to price the floor described above.

Display the binomial tree of prices.

In [37]:
refratetree = compound * (np.exp(ratetree / compound)-1)
format_bintree(refratetree.iloc[:tsteps,:tsteps], style='{:.2%}')

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,5.22%,5.43%,5.58%,5.74%,5.61%,5.99%,6.54%,7.31%,7.94%,9.34%,11.13%,13.25%
1,,4.71%,4.85%,4.98%,4.87%,5.20%,5.67%,6.35%,6.89%,8.10%,9.65%,11.49%
2,,,4.05%,4.16%,4.07%,4.34%,4.74%,5.30%,5.76%,6.76%,8.06%,9.59%
3,,,,3.35%,3.28%,3.50%,3.82%,4.27%,4.64%,5.44%,6.49%,7.71%
4,,,,,2.30%,2.45%,2.68%,2.99%,3.25%,3.81%,4.54%,5.40%
5,,,,,,1.68%,1.83%,2.05%,2.22%,2.61%,3.11%,3.69%
6,,,,,,,1.24%,1.38%,1.50%,1.76%,2.09%,2.49%
7,,,,,,,,0.93%,1.01%,1.18%,1.40%,1.67%
8,,,,,,,,,0.68%,0.79%,0.95%,1.12%


In [38]:
payoff = lambda r: N * dt * np.maximum(STRIKE-r,0)

cftree = payoff(refratetree.iloc[:tsteps,:tsteps])

### no floorlet until T=1, so ensure 0 until then
cftree.loc[0,0] = 0

format_bintree(cftree)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,,,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,,,,0.19,0.21,0.15,0.07,0.00,0.00,0.00,0.00,0.00
4,,,,,0.45,0.41,0.36,0.28,0.21,0.07,0.00,0.00
5,,,,,,0.61,0.57,0.51,0.47,0.37,0.25,0.10
6,,,,,,,0.72,0.68,0.65,0.59,0.50,0.40
7,,,,,,,,0.79,0.77,0.73,0.67,0.61
8,,,,,,,,,0.86,0.83,0.79,0.74


In [39]:
format_bintree(bintree_pricing(payoff=payoff, ratetree=ratetree.iloc[:tsteps,:tsteps], undertree= refratetree.iloc[:tsteps,:tsteps], cftree=cftree, timing='deferred'),style='{:.3f}')

c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\binomial.py:129: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  val_avg = pstars[t] * valuetree.iloc[state,-steps_back] + (1-pstars[t]) * valuetree.iloc[state+1,-steps_back]


time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,1.597,1.022,0.578,0.275,0.103,0.026,0.003,0.000,0.000,0.000,0.000,0.000
1,,2.214,1.494,0.898,0.455,0.182,0.050,0.006,0.000,0.000,0.000,0.000
2,,,2.986,2.127,1.363,0.739,0.320,0.095,0.012,0.000,0.000,0.000
3,,,,3.880,2.935,1.997,1.174,0.551,0.181,0.025,0.000,0.000
4,,,,,4.514,3.509,2.552,1.677,0.933,0.342,0.050,0.000
5,,,,,,4.669,3.684,2.748,1.891,1.114,0.496,0.101
6,,,,,,,4.482,3.518,2.607,1.750,1.000,0.401
7,,,,,,,,4.040,3.093,2.183,1.344,0.606
8,,,,,,,,,3.418,2.471,1.573,0.743


## 2.4.

What do you think is the most relevant reason for the difference in pricing between Black's formula and BDT?

**ANSWER:** 

The difference most likely comes from BDT being a discrete model opposed to Black's formula being a continuous model. If the steps of the BDT converged to nearly 0 then these models would yield the same result.

## 2.5.

Use the BDT tree to price a swap...
* `receiving-fixed` 
* swap rate of `3.65%`
* tenor `T=3`
* notional `N=100`.
* frequency quarterly (`n=4`)

Display the pricing tree.

In [40]:
T=3
dt = 1/4
N = 100
tsteps = int(T/dt)
STRIKE = .0365
payoff = lambda r: N * dt * (STRIKE-r)

In [41]:
swaptree = bintree_pricing(payoff=payoff, ratetree=ratetree.iloc[:tsteps,:tsteps], undertree= refratetree, cftree=cftree, cfdelay=True)
format_bintree(swaptree)

c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\binomial.py:129: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  val_avg = pstars[t] * valuetree.iloc[state,-steps_back] + (1-pstars[t]) * valuetree.iloc[state+1,-steps_back]


time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,1.35,0.71,0.17,-0.26,-0.59,-0.86,-1.10,-1.33,-1.57,-1.80,-2.06,-2.32
1,,2.03,1.27,0.60,0.06,-0.34,-0.65,-0.90,-1.14,-1.39,-1.64,-1.90
2,,,2.84,1.96,1.16,0.47,-0.05,-0.41,-0.68,-0.93,-1.20,-1.45
3,,,,3.76,2.81,1.85,0.99,0.32,-0.15,-0.45,-0.70,-1.00
4,,,,,4.40,3.40,2.43,1.55,0.79,0.15,-0.22,-0.43
5,,,,,,4.56,3.57,2.64,1.78,1.00,0.38,-0.01
6,,,,,,,4.37,3.41,2.50,1.64,0.89,0.29
7,,,,,,,,3.93,2.98,2.07,1.23,0.49
8,,,,,,,,,3.31,2.36,1.46,0.63


***

# 3. Callable Bond

#### Note:
Continue with the rate curves and BDT model provided in `rate curves` and `rate tree` in the provided spreadsheet.

## 3.1.

Consider a bond with:
* `T=3`
* face value of `N=100`
* coupons at `quarterly` frequency
* annualized coupon rate of `cpn=6%`.

Use your BDT tree to price this bond and display the pricing tree.

In [42]:
curves

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052226,0.052226,0.987112,NaN,NaN,0.140538
0.50,0.051470,0.051465,0.974756,0.050705,0.140538,0.140538
0.75,0.050427,0.050411,0.963125,0.048303,0.161925,0.178617
1.00,0.049243,0.049209,0.952268,0.045606,0.183311,0.215305
1.25,0.047534,0.047467,0.942722,0.040506,0.233815,0.353890
1.50,0.046112,0.046016,0.933672,0.038769,0.272932,0.377696
1.75,0.044914,0.044793,0.925009,0.037462,0.302241,0.393585
2.00,0.043913,0.043770,0.916617,0.036620,0.323320,0.398881
2.25,0.042897,0.042729,0.908800,0.034407,0.337748,0.395042


In [43]:
ratetree

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75
state,,,,,,,,,,,,
0,0.051888,0.053922,0.055442,0.056946,0.055674,0.059412,0.064853,0.072473,0.078660,0.092281,0.109781,0.130351
1,NaN,0.046853,0.048173,0.049480,0.048375,0.051623,0.056350,0.062971,0.068347,0.080182,0.095388,0.113260
2,NaN,NaN,0.040293,0.041386,0.040462,0.043179,0.047133,0.052671,0.057167,0.067066,0.079785,0.094734
3,NaN,NaN,NaN,0.033370,0.032624,0.034815,0.038003,0.042468,0.046094,0.054075,0.064330,0.076384
4,NaN,NaN,NaN,NaN,0.022901,0.024438,0.026676,0.029811,0.032356,0.037958,0.045157,0.053618
5,NaN,NaN,NaN,NaN,NaN,0.016751,0.018285,0.020433,0.022178,0.026018,0.030952,0.036752
6,NaN,NaN,NaN,NaN,NaN,NaN,0.012336,0.013785,0.014962,0.017553,0.020881,0.024794
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009251,0.010041,0.011779,0.014013,0.016638
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006764,0.007935,0.009440,0.011209


**Vanilla Bond - Call Option on Vanilla Bond**

In [44]:
T = 3
N = 100
FREQ = 4
cpn = 0.06
STRIKE = 100
compound = 4
dt = 1/compound
tsteps = int(T/dt)

wrapper_bond = lambda r: payoff_bond(r, dt, facevalue=N * (1+cpn/FREQ))

In [45]:
cftree = construct_bond_cftree(T, compound, cpn)
cftree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
1,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
2,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
3,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
4,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
5,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
6,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
7,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0
8,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0


In [46]:
bondtree = bintree_pricing(payoff=wrapper_bond, ratetree=ratetree.iloc[:tsteps,:tsteps], cftree=cftree)
bondtree

c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\binomial.py:129: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  val_avg = pstars[t] * valuetree.iloc[state,-steps_back] + (1-pstars[t]) * valuetree.iloc[state+1,-steps_back]


time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75
state,,,,,,,,,,,,
0,103.895129,103.979013,101.160152,101.396999,98.747325,99.109985,96.672805,97.448647,95.556902,96.915879,95.790592,98.245665
1,NaN,106.524261,103.620252,103.747088,100.954411,101.152708,98.513317,99.057271,96.903776,97.993398,96.564897,98.666324
2,NaN,NaN,105.938413,106.004354,103.122411,103.212748,100.419912,100.764623,98.354319,99.154165,97.390198,99.124368
3,NaN,NaN,NaN,108.017554,105.091253,105.128915,102.245948,102.455730,99.846146,100.385995,98.271119,99.580154
4,NaN,NaN,NaN,NaN,106.753649,106.774854,103.849889,103.988246,101.252457,101.620756,99.233505,100.148530
5,NaN,NaN,NaN,NaN,NaN,107.958308,105.008512,105.101323,102.279803,102.528849,99.945866,100.571698
6,NaN,NaN,NaN,NaN,NaN,NaN,105.814203,105.877935,102.999376,103.168081,100.449984,100.872800
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.404121,103.487519,103.602650,100.793608,101.078676
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.813452,103.892575,101.022766,101.215982


## 3.2. (7pts)

Suppose the bond is callable by the issuer.

* `European` style
* expiration of `Topt=1.5`
* (clean) `strike=100`

What is the value of the issuer's call option?

*Note that there is no difference between clean or dirty strike given that the bond pays coupons quarterly, and we are looking at quarterly steps in the tree.*

In [47]:
TOPT = 1.5
STRIKE = 100

payoff_call = lambda p: np.maximum(p-STRIKE,0)

In [48]:
accint = construct_accint(bondtree.columns.values, compound, cpn)
cleantree = np.maximum(bondtree.subtract(accint,axis=1),0)
cleantree

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75
state,,,,,,,,,,,,
0,103.895129,102.479013,101.160152,99.896999,98.747325,97.609985,96.672805,95.948647,95.556902,95.415879,95.790592,96.745665
1,NaN,105.024261,103.620252,102.247088,100.954411,99.652708,98.513317,97.557271,96.903776,96.493398,96.564897,97.166324
2,NaN,NaN,105.938413,104.504354,103.122411,101.712748,100.419912,99.264623,98.354319,97.654165,97.390198,97.624368
3,NaN,NaN,NaN,106.517554,105.091253,103.628915,102.245948,100.955730,99.846146,98.885995,98.271119,98.080154
4,NaN,NaN,NaN,NaN,106.753649,105.274854,103.849889,102.488246,101.252457,100.120756,99.233505,98.648530
5,NaN,NaN,NaN,NaN,NaN,106.458308,105.008512,103.601323,102.279803,101.028849,99.945866,99.071698
6,NaN,NaN,NaN,NaN,NaN,NaN,105.814203,104.377935,102.999376,101.668081,100.449984,99.372800
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.904121,103.487519,102.102650,100.793608,99.578676
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.813452,102.392575,101.022766,99.715982


In [49]:
calltree = bintree_pricing(payoff=payoff_call, ratetree=ratetree.iloc[:tsteps,:tsteps], undertree= cleantree)
format_bintree(calltree)

c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\binomial.py:129: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  val_avg = pstars[t] * valuetree.iloc[state,-steps_back] + (1-pstars[t]) * valuetree.iloc[state+1,-steps_back]


time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75
state,,,,,,,,,,,,
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,,,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,,,,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,,,,,,,0.00,0.00,0.00,0.00,0.00,0.00
7,,,,,,,,0.00,0.00,0.00,0.00,0.00
8,,,,,,,,,0.00,0.00,0.00,0.00


## 3.3. (3pts)

What is the value of the callable bond?

In [50]:
# I am not sure why my call tree has all 0s but this would indicate that the value of the callable bond is 0
cleantree.iloc[0, 0]


103.89512882849785

## 3.4.

We found that the Freddie Mac callable bonds often have negative option-adjusted spreads (OAS). 

Why was this? Do you expect that the european callable bond would be less prone to this phenomenon?

**ANSWER:** European callable bonds would be less prone to this phenomenon. This is because European callable bonds only have one callable date whereas american callable bonds can be called at multiple dates allowing for more optionality and thus you have to pay for this and so there is a larger negative OAS spread than euro callable bonds would have.

## 3.5.

We found that the Freddie Mac american callable bond never priced above 100.

Does this bond ever price above 100? Explain why this is possible when it was not for the Freddie Mac callable.

**ANSWER:**

Yes, this bond does price over 100. This is possible because after the one and only call date then the bond can have a price over 100 and function like a vanilla bond.

## 3.6.

Price the callable bond without using binomial trees.

* Use standard closed-form pricing for the vanilla bond, given the rate curve data.
* Use Black's formula to price the callable option.

Report this newly modeled price.

*Note: In Black's formula, use the flat volatility for the option term.*

## 3.7.

Which assumptions of Black's formula do we prefer to Black-Scholes for this problem?

## 3.8.

Which aspects of the calculation in `3.6` differ because we are using Black's formula, not Black-Scholes? 

*Note: Unlike `3.7`, we're asking about the differences of implementing the calculation, not the differences in the assumptions of the models.

## 3.9. 

Suppose that we buy the callable bond, but we want to hedge against it being called by the issuer. That is to say, we want to retain upside exposure to rates decreasing while otherwise keeping the nature of the position the same.

Explain specifically how you would use caps, floors, or swaptions to achieve this. 

No need to calculate the value; rather, just describe the specific product you would go long (or short) and how it would transform your exposure.

## 3.10.

How would your answer to `3.9` change if it were a callable bond with **american** exercise by the issuer?

***

# 4. Swaptions and SABR

## 4.1.

Use the market data in `rate curves` to calculate the relevant forward swap rate for an at-the-money (ATM) swaption with...
* `expiration=1`
* `tenor=2`

Report this forward swap rate.

In [51]:
curves

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052226,0.052226,0.987112,NaN,NaN,0.140538
0.50,0.051470,0.051465,0.974756,0.050705,0.140538,0.140538
0.75,0.050427,0.050411,0.963125,0.048303,0.161925,0.178617
1.00,0.049243,0.049209,0.952268,0.045606,0.183311,0.215305
1.25,0.047534,0.047467,0.942722,0.040506,0.233815,0.353890
1.50,0.046112,0.046016,0.933672,0.038769,0.272932,0.377696
1.75,0.044914,0.044793,0.925009,0.037462,0.302241,0.393585
2.00,0.043913,0.043770,0.916617,0.036620,0.323320,0.398881
2.25,0.042897,0.042729,0.908800,0.034407,0.337748,0.395042


### I assume frequency of swap is 1

In [53]:
expry = 1
tenor = 2
Topt = expry
Tswap = Topt+tenor
fwdrate = curves['forwards'][Topt]
freqswap = 1
fwdswap = calc_fwdswaprate(curves['discounts'], Topt, Tswap, freqswap=freqswap)
fwdswap

0.03715209929137732

## 4.2.

#### Note

Regardless of what you calculated in the previous problem **use a forward swap rate of `.0365` for the rest of this section**, not just this specific question.

Don't worry that the forward swap rate provided here may not match your answer from the previous section. We are using it to ensure all solutions below are based on the same forward swap rate.

In [54]:
fwdswap = 0.0365

#### Continuing...

Use the data in `volskew` which gives market quotes (in terms of Black vol) on swaptions across various strikes. 
* The strikes listed are relative to the ATM strike, which equals the forward swap rate given to you in the previous paragraph.
* All these quotes are for the same expiration and tenor considered in `4.1`.

Report the price of the swaption with specification of...
* struck at the money
* notional of `100`
* a `receiver` swaption

In [59]:
STRIKE = volskew.loc[0, 0]
VOL = curves.loc[Topt, 'fwd vols']
print(VOL)
print(STRIKE)
volskew

0.2153045538622541
34.8


,reference,instrument,model,date,expiration,tenor,-300,-200,-100,-50,-25,0,25,50,100,200,300
0,SOFR,swaption,black,2024-03-05,1,2,100.35,57.7,42.28,37.82,36.15,34.8,33.74,32.93,31.94,31.56,32.14


In [61]:
# receiver means receive fixed, pay floating and so we are short rates
blacks_formula(Topt, VOL, STRIKE, fwdswap, isCall=False)

34.7635

## 4.3. (10pts)

Use SABR to consider pricing for strikes which are not listed. Sepcifically,

* Input the SABR parameters given in `sabrparams` in the exam data sheet to get the implied volatility for any strike.

* To do this, choose the `SLIM` SABR model, where $\alpha$ (also called $\sigma_0$) is a function of the other SABR parameters. Thus, you won't make use of the estimated $\alpha$ parameter; rather, you'll make use of the other parameters in conjunction with the ATM market quoted vol.

Consider a `STRIKE=.03`.

Report
* the SABR-implied vol for this strike, (and the provided forward swap rate.) 
* the price for this swaption given by Black's formula.

In [63]:
STRIKE = 0.03
sabrparams

,estimate
parameter,
beta,0.250000
alpha,0.028011
nu,0.689461
rho,-0.160282


In [85]:
volATM = VOL
strikes = [STRIKE]
vols = volskew.iloc[:,6:]
BETA = sabrparams.loc['beta']
doSLIM = True
F = fwdswap-1e-8
print(volATM)
print(strikes)
print(vols)
print(BETA)

0.2153045538622541
[0.03]
     -300  -200   -100    -50    -25     0     25     50    100    200    300
0  100.35  57.7  42.28  37.82  36.15  34.8  33.74  32.93  31.94  31.56  32.14
estimate    0.25
Name: beta, dtype: float64


In [86]:
def obj_fun(xargs):
    nu = xargs[0]
    rho = xargs[1]
    alpha = xargs[2]
    
    ivolSABR = np.zeros(len(strikes))
    
    for i,strike in enumerate(strikes):
         ivolSABR[i] = sabr(BETA,nu,rho,alpha,F,strike,Topt)
    
    error = ((ivolSABR - vols.values)**2).sum()
    
    return error

def obj_fun_slim(xargs):
    nu = xargs[0]
    rho = xargs[1]
    ivolSABR = np.zeros(len(strikes))
    
    for i,strike in enumerate(strikes):
         ivolSABR[i] = sabr_slim(BETA,nu,rho,F,strike,Topt,volATM)
    
    error = ((ivolSABR - vols.values)**2).sum()
    
    return error

if not doSLIM:
    x0 = np.array([.6,0,.1])
    fun = obj_fun
else:
    fun = obj_fun_slim
    x0 = np.array([.6,0,.1])

optim = minimize(fun,x0)
xstar = optim.x
nustar = xstar[0]
rhostar = xstar[1]
   
if doSLIM:
    alphastar = solve_alpha(BETA,nustar,rhostar,Topt,volATM,F)
    ivolSABR = sabr_slim(BETA,nustar,rhostar,F,strikes,Topt,volATM)
else:
    alphastar = xstar[2]
    ivolSABR = sabr(BETA,nustar,rhostar,alphastar,F,strikes,Topt)
    
error = optim.fun

c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\volskew.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  coefs[3] = (1-beta)**2 * T / (24*f**(2-2*beta))
c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\volskew.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  coefs[2] = rho * beta * nu * T / (4*f**(1-beta))
c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\volskew.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  coefs[0] = -volATM * f**(1-beta)
C:\Users\alexa\AppData\Local\Temp\ipykernel_20620\3463512740.py:21: FutureWarning: Calling fl

C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\volskew.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  coefs[3] = (1-beta)**2 * T / (24*f**(2-2*beta))
c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\volskew.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  coefs[2] = rho * beta * nu * T / (4*f**(1-beta))
c:\Users\alexa\OneDrive - The University of Chicago\FINM_37500\FINM_37500_Assignments\exam\../cmds\volskew.py:31:

TypeError: can't multiply sequence by non-int of type 'float'

## 4.4. 

Suppose the forward swap rate changes by `+10bps`.

(We are considering an instantaneous change, so no need to change the time-to-expiration.)

Report
* the new price, assuming vol stays constant
* the new vol and the new price, assuming vol responds according to the SABR specification of part `4.3`.

In [87]:
SHIFT = 50

In [88]:
idloc = (sabrcurve.index.to_series()-fwdswap).abs().idxmin()

newvols = sabrcurve.loc[idloc]
strikeATM = strikes[idstrikeATM]

Frange = F + [0,SHIFT]

NameError: name 'sabrcurve' is not defined

## 4.5.

Use these new prices to calculate and report the approximate delta, for both a static and dynamic vol.

Specifically, calculate the numerical change in price per change in forward rate when...
* holding vol constant
* modeling vol changing according to SABR

How much does the "augmented" delta differ from the classic delta?

## 4.6.

Without doing any new calculation, what do you think the effect of a SABR model would be on delta for a payer-swaption in response to a decrease in interest rates?

* Would the delta be positive or negative?
* Would the "augmented" effects of SABR cause the price to be larger or smaller than what the classic delta would imply? 

**ANSWER** If rates decrease then the delta for the payer would increase ebcause they are paying floating and so a decrease in rates wouold cause the value of the swaption to increase and be more sensitive to further decreases in interest rates.

## 4.7.

Consider again the given forward swap rate provided in `4.2` (without the shift considered in part `4.4`.)

Use the SABR vol skew to price the swaption (same expiry and tenor) struck `-300bps` OTM. 

* Report this model price and compare it to the market quote at -300bps.

* Why might we trust the model price more than the provided market quote here?

**ANSWER** Market quotes can be unreliable and bloomberg uses an interpolation processs that is not as sophiscated as conducting modeling or using modeling that market makers use. Therefore the model price is more trustowrthy than the provided market quote.

## 4.8.

What advantage does SABR have over local vol models? 

Be specific.

**ANSWER:** 

Local vol models fit a function for the volatility to exactly the implied vols quoted by the market. SABR has the advantage of not overfitting the vol curve like local vol models do and instead focusing on the dynamics of the vol instead of exactly matching it to the market which results in an inocrrect and overfit vega.

## 4.9.

For the estimated SABR model in the picture, is ATM implied volatility higher or lower as the interest rate goes higher?

In a sentence, describe specifically what vol path indicates to us about this option market.

(If the figure is not rendering in this cell, find it in `../data/volpath_example.png`.

![title](../data/volpath_example.png)

**ANSWER:** As interest rates go higher the implied goes higher. The vol path tells us that for the 

## 4.10.

How do we quantify the **vol path** in SABR? Is it estimated or assumed? Be specific.

***